# Hands-on 1: Ingestion and Chunking

## Problem

DEFINITION:

https://docs.llamaindex.ai/en/stable/examples/retrievers/relative_score_dist_fusion/

## Code

In [52]:
# if running on colab uncomment the those lines
# %pip install llama-index>=0.12.2
# %pip install llama-index-retrievers-bm25>=0.50
# !mkdir -p 'data/paul_graham/'
# !wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt' -O 'data/paul_graham/paul_graham_essay.txt'

In [53]:
# load .env
from dotenv import load_dotenv
load_dotenv()

# import os
# os.environ["OPENAI_API_KEY"] = "sk-proj-..."

True

In [54]:
from llama_index.core import SimpleDirectoryReader
import nest_asyncio

nest_asyncio.apply()

documents = SimpleDirectoryReader("./data/paul_graham").load_data()

In [55]:

from llama_index.core import VectorStoreIndex
from llama_index.core.node_parser import SentenceSplitter
from rich import print
from llama_index.retrievers.bm25 import BM25Retriever
from llama_index.core.retrievers import QueryFusionRetriever
from llama_index.core.memory import ChatMemoryBuffer

In [56]:
QUESTION = "Why the author was in Florance?"

In [57]:
splitter = SentenceSplitter(chunk_size=256)

index = VectorStoreIndex.from_documents(
    documents, transformations=[splitter], show_progress=True
)

vector_retriever = index.as_retriever(similarity_top_k=3)
bm25_retriever = BM25Retriever.from_defaults(
    docstore=index.docstore, similarity_top_k=3
)


Generating embeddings: 100%|██████████| 587/587 [00:09<00:00, 58.80it/s]


In [58]:
QUERY_GEN_PROMPT="""You are a helpful assistant that generates multiple search queries based on a single input query. Generate {num_queries} search queries, one on each line, related to the following input query:
Query: {query}
Queries:"""

In [59]:
retriever = QueryFusionRetriever(
    [vector_retriever, bm25_retriever],
    similarity_top_k=10,
    num_queries=3,
    mode="reciprocal_rerank",
    use_async=True,
    verbose=True,
    query_gen_prompt=QUERY_GEN_PROMPT,
)

In [60]:
CHAT_PROMPT = """You're a chatbot, able to have normal conversations, about an essay discussing Paul Grahams life. 
Here are the relevant documents for the context:
{context_str}
Instruction: use the previous chat history, or the context aboute, to interact with the user."""

In [61]:
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.legacy.chat_engine.condense_question import CondenseQuestionChatEngine

memory = ChatMemoryBuffer.from_defaults(token_limit=3000)
query_engine = RetrieverQueryEngine(retriever)

chat_engine = CondenseQuestionChatEngine.from_defaults(
    query_engine=query_engine,
    memory=memory,
    context_prompt=CHAT_PROMPT,
)

In [62]:
chat_engine.chat("What did Paul Graham do in the summer of 1995?")

Generated queries:
1. What projects did Paul Graham work on in the summer of 1995?
2. Did Paul Graham write any influential essays or articles in the summer of 1995?


AgentChatResponse(response='In the summer of 1995, Paul Graham organized a Summer Founders Program where undergraduates were invited to apply to start startups instead of taking temporary jobs at tech companies.', sources=[ToolOutput(content='In the summer of 1995, Paul Graham organized a Summer Founders Program where undergraduates were invited to apply to start startups instead of taking temporary jobs at tech companies.', tool_name='query_engine', raw_input={'query': 'What did Paul Graham do in the summer of 1995?'}, raw_output=Response(response='In the summer of 1995, Paul Graham organized a Summer Founders Program where undergraduates were invited to apply to start startups instead of taking temporary jobs at tech companies.', source_nodes=[NodeWithScore(node=TextNode(id_='f40efdfc-d34e-4ee9-9fb3-132200212afd', embedding=None, metadata={'file_path': 'c:\\Users\\n.fretti\\Desktop\\projects\\rag_and_roll\\lesson_02\\data\\paul_graham\\paul_graham_essay.txt', 'file_name': 'paul_graha

In [63]:
chat_engine.chat("Why did he organize it?")

Generated queries:
1. What were the motivations behind Paul Graham's decision to launch the Summer Founders Program in 1995?
2. How did the Summer Founders Program organized by Paul Graham in 1995 impact the startup ecosystem and the participating undergraduates?


AgentChatResponse(response='Paul Graham organized the Summer Founders Program in the summer of 1995 to provide undergraduates with a more interesting summer experience than working at companies like Microsoft.', sources=[ToolOutput(content='Paul Graham organized the Summer Founders Program in the summer of 1995 to provide undergraduates with a more interesting summer experience than working at companies like Microsoft.', tool_name='query_engine', raw_input={'query': 'Why did Paul Graham organize a Summer Founders Program in the summer of 1995 where undergraduates were invited to apply to start startups instead of taking temporary jobs at tech companies?'}, raw_output=Response(response='Paul Graham organized the Summer Founders Program in the summer of 1995 to provide undergraduates with a more interesting summer experience than working at companies like Microsoft.', source_nodes=[NodeWithScore(node=TextNode(id_='7a67d186-814d-46c2-8d30-713065b61965', embedding=None, metadata={'file_pat

In [64]:
chat_engine.reset()

In [65]:
response = chat_engine.chat("What did I ask you before?")

Generated queries:
1. How can I retrieve the conversation history with a virtual assistant?
2. Is there a way to access past queries and responses from a chatbot conversation?
